In [ ]:
import numpy as np

You are part of a team working to make mobile payments available globally, and are asked to build a deep learning model to detect fraud
--whenever someone makes a payment, you want to see if the payment might be fraudulent, 
such as if the user's account has been taken over by a hacker.

You already know that backpropagation is quite challenging to implement, and sometimes has bugs. 
Because this is a mission-critical application, your company's CEO wants to be really certain that your implementation of backpropagation 
is correct. Your CEO says, "Give me proof that your backpropagation is actually working!" To give this reassurance, you are going to use
"gradient checking."

Let's do it!

In [ ]:
#first , lets define some helper functions
def sigmoid(x):
  
    s = 1 / (1 + np.exp(-x))
    return s

def relu(x):
 
    s = np.maximum(0, x)
    
    return s

def dictionary_to_vector(parameters):
    """
    Roll all our parameters dictionary into a single vector satisfying our specific required shape.
    """
    keys = []
    count = 0
    for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        
        # flatten parameter
        new_vector = np.reshape(parameters[key], (-1, 1))
        keys = keys + [key] * new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta, keys

def vector_to_dictionary(theta):
    """
    Unroll all our parameters dictionary from a single vector satisfying our specific required shape.
    """
    parameters = {}
    parameters["W1"] = theta[: 20].reshape((5, 4))
    parameters["b1"] = theta[20: 25].reshape((5, 1))
    parameters["W2"] = theta[25: 40].reshape((3, 5))
    parameters["b2"] = theta[40: 43].reshape((3, 1))
    parameters["W3"] = theta[43: 46].reshape((1, 3))
    parameters["b3"] = theta[46: 47].reshape((1, 1))

    return parameters

def gradients_to_vector(gradients):
    """
    Roll all our gradients dictionary into a single vector satisfying our specific required shape.
    """
    
    count = 0
    for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
        # flatten parameter
        new_vector = np.reshape(gradients[key], (-1, 1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta

In [ ]:
def forward_propagation(x, theta):
    """
    Implement the linear forward propagation (compute J) presented in Figure 1 (J(theta) = theta * x)

    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well

    Returns:
    J -- the value of function J, computed using the formula J(theta) = theta * x
    """

    J = theta * x
    return J

x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

In [ ]:
def backward_propagation(x, theta):
    """
    Computes the derivative of J with respect to theta (see Figure 1).

    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well

    Returns:
    dtheta -- the gradient of the cost with respect to theta

    J(θ) = θ × x
     The derivative of θ with respect to itself is 1
     x is treated as a constant (the input)
     So: dJ/dθ = x × 1 = x
    """
    dtheta = x
    return dtheta

x, theta = 3, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

In [ ]:
def gradient_check(x, theta, epsilon=1e-7, print_msg=False):
    """
    Arguments:
    x -- a float input
    theta -- our parameter, a float as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient. Float output
    """
 
    # Compute gradapprox using right side of formula (1). epsilon is small enough, you don't need to worry about the limit.
    # Step 1
    theta_plus  = theta + epsilon
    # Step 2
    theta_minus = theta - epsilon
    # Step 3
    J_plus      = forward_propagation(x, theta_plus)
    # Step 4
    J_minus     = forward_propagation(x, theta_minus)
    # Step 5
    gradapprox  = (J_plus - J_minus) / (2 * epsilon)
    
    # Check if gradapprox is close enough to the output of backward_propagation()
    grad = backward_propagation(x, theta)
    
    # Step 1'
    numerator   = np.linalg.norm(grad - gradapprox)
    # Step 2'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    # Step 3'
    difference  = numerator / denominator
    
    
    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [ ]:
x, theta = 3, 4
difference = gradient_check(x, theta, print_msg=True)

# N-Dimensional Gradient Checking

In [ ]:
def forward_propagation_n(X, Y, parameters):
    """

    Arguments:
    X -- training set for m examples
    Y -- labels for m examples
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)

    Returns:
    cost -- the cost function (logistic cost for m examples)
    cache -- a tuple with the intermediate values (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    """

    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)

    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    return cost, cache

In [ ]:
def backward_propagation_n(X, Y, cache):
    """
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()

    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """

    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T) * 2
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}

    return gradients

In [ ]:
def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7, print_msg=False):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output by forward_propagation_n

    Arguments:
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
    grad -- output of backward_propagation_n, contains gradients of the cost with respect to the parameters
    X -- input datapoint, of shape (input size, number of examples)
    Y -- true "label"
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)

    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """

    # Set-up variables
    parameters_values, _ = dictionary_to_vector(parameters)

    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))

    # Compute gradapprox
    for i in range(num_parameters):


        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = None
        # "_" is used because the function you have outputs two parameters but we only care about the first one
        # Compute J_plus[i]
        # Step 1
        theta_plus = np.copy(parameters_values)
        # Step 2
        theta_plus[i][0] = theta_plus[i][0] + epsilon
        parameters_plus = vector_to_dictionary(theta_plus) 
        # Step 3
        J_plus[i], _ = forward_propagation_n( X, Y, parameters_plus,)

        # Compute J_minus[i]
        # Step 1
        theta_minus = np.copy(parameters_values)
        # Step 2
        theta_minus[i][0] = theta_minus[i][0] - epsilon
        parameters_minus = vector_to_dictionary(theta_minus)
        # Step 3
        J_minus[i], _ = forward_propagation_n( X, Y, parameters_minus,)

        # Compute gradapprox[i]
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

       

    # Compare gradapprox to backward propagation gradients by computing difference.
    # Step 1'
    numerator = np.linalg.norm(grad - gradapprox)

    # Step 2'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)


    # Step 3'
    difference = numerator / denominator


    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

    return difference

In [ ]:
#Lets test it
np.random.seed(1)
x = np.random.randn(4,3)
y = np.array([1, 1, 0])
W1 = np.random.randn(5,4) 
b1 = np.random.randn(5,1) 
W2 = np.random.randn(3,5) 
b2 = np.random.randn(3,1) 
W3 = np.random.randn(1,3) 
b3 = np.random.randn(1,1) 
parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3 }

cost, cache = forward_propagation_n(x, y, parameters)
gradients = backward_propagation_n(x, y, cache)
difference = gradient_check_n(parameters, gradients, x, y, 1e-7, True)

#Well, looks like there were error in the backward_propagation_n 

The code contains two deliberate "bugs" :

Line dW2: It is being multiplied by 2, which is incorrect.
Line db1: It is dividing 4. / m, but it should be 1. / m.
    
Here is the corrected code:

In [ ]:
def backward_propagation_n(X, Y, cache):
    """
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    # FIX 1: Removed "* 2" at the end
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    # FIX 2: Changed "4. / m" to "1. / m"
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
# Not Lets test again

np.random.seed(1)
x = np.random.randn(4,3)
y = np.array([1, 1, 0])
W1 = np.random.randn(5,4) 
b1 = np.random.randn(5,1) 
W2 = np.random.randn(3,5) 
b2 = np.random.randn(3,1) 
W3 = np.random.randn(1,3) 
b3 = np.random.randn(1,1) 
parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3 }

cost, cache = forward_propagation_n(x, y, parameters)
gradients = backward_propagation_n(x, y, cache)
difference = gradient_check_n(parameters, gradients, x, y, 1e-7, True)